In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\jyravi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [197]:
import requests
import json
import time


#### Your speech service region and the subscription_key

In [198]:
# Your subscription key and region for the speech service
SERVICE_REGION = ""

NAME = "Simple transcription"
DESCRIPTION = "Simple transcription description"

#LOCALE = "en-IN"
SUBSCRIPTION_KEY=" "

In [199]:
from http import HTTPStatus

expected_status_codes=[HTTPStatus.CREATED]
expected_status_codes

[<HTTPStatus.CREATED: 201>]

#### input audio in Azure blob and defining destination url for storing the transcribed file

In [152]:

destcontainerurl="" ### paste your Blob sas URL
uri=f"https://{SERVICE_REGION}.api.cognitive.microsoft.com/speechtotext/v3.2/transcriptions"

In [1]:
Sas_token=destcontainerurl.split("?")[-1]

##### diarization is possible only for mono audios.

In [203]:
content ={ "contentUrls": [
    
    "" # paste your url
  ],
  "locale": "en-IN",
  "displayName": "My Transcription",
  #"model": Null,
"properties": {
    #"wordLevelTimestampsEnabled": True,
    "diarizationEnabled": True,
    "timeToLive" : "PT30M",
    "destinationContainerUrl":destcontainerurl,
    "languageIdentification": {
      "candidateLocales": [
        "en-IN","hi-IN"
      ],
    }
  },}

In [2]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
response = requests.post(uri, headers=headers, json=content)
if response.status_code not in expected_status_codes :
    raise Exception(f"The POST request to {uri} returned a status code {response.status_code} that was not in the expected status codes: {expected_status_codes}")
else :
    try :
        response_json = response.json()
        print({ "headers" : response.headers, "text" : response.text, "json" : response_json })
    except Exception :
        { "headers" : response.headers, "text" : response.text, "json" : None }

In [2]:
response_json

#### Get the status of the transcription

In [208]:
rr=requests.get(response_json['self'],headers=headers)

In [3]:
rr.json()

#### When 'status' == 'Succeeded'

In [4]:
response_json['links']['files']

#### Get the transcribed file

In [211]:
r=requests.get(response_json['links']['files'],headers=headers)

In [212]:
result_json=r.json()

In [5]:
result_json

In [6]:
result_json['values'][0]['links']['contentUrl']

#### Results extraction when destination url is not defined.

##### Here we have taken the only the first ContentUrl, as we have only given one URL. This will go into a loop when you pass multiple Urls as a list.

In [3]:
result_wo_sas=requests.get(result_json['values'][0]['links']['contentUrl'])

In [4]:
result_wo_sas.json()

### sas_token for the destination_url

In [215]:
Sas_token= 

#### constructing the url with sas_token for reading the file from destination url
##### Please create loop for reading through all the contentUrls if a list of URLs are provided for transcribing

In [7]:
file_uri=result_json['values'][0]['links']['contentUrl']+"?"+Sas_token
file_uri

In [217]:
result=requests.get(url=file_uri)

In [8]:
result.json()

In [219]:
result.json().keys()

dict_keys(['source', 'timestamp', 'durationInTicks', 'duration', 'combinedRecognizedPhrases', 'recognizedPhrases'])

#### Below step is one way to combine the speaker and the utterance. There could be other ways,please feel free to make those according how the the use wants the output to look like

In [220]:
conversation=[]
for i in result.json()['recognizedPhrases']:
    if i["recognitionStatus"]=='Success':
        #print('speaker '+ str(i['speaker'])+":")
        #print(i['nBest'][0]['lexical'])
        conversation.append(['speaker '+ str(i['speaker'])+": "+ i['nBest'][0]['lexical']])


In [5]:
conversation

### Passing "conversation" dynamically to AOAI.

##### Please make necessary changes in the system prompt to suit your use cases

In [231]:
import os
import requests
import base64

# Configuration
API_KEY = ""
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request
payload = { "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": '''You are AI assistant for analyzing transcripts between customer support agent and the customer. 
          You will be provided with the conversation transcripts with speakers diarized as speaker 1 and speaker 2.
          1. Identify agent and customer from the conversation flow.
          2.  Summarize the issue/query/concern from the customer?
          3. Summarize the response provided by the agent.
          4. Was the customer satisfied with the resolution provided by the agent?
          5.Overall sentiment at the end of the call.
          Please respond with the answers in the below json format
          "{"Speaker 1": "",
          "Speaker 2": " ",
          "Issue" : " ",
          "Resolution":"",
          "Customer Satified" : " ",
          "Overall Sentiment": " "}"'''
          
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{conversation}"
        }
      ]
    },],  
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800}

ENDPOINT = f"https://{}.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"



In [9]:
# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

In [6]:
print(response.json()['choices'][0]['message']['content'])